# 環境設定

configファイルにパスを通す。パスの参照順は以下の通り。
Jupyterから特定のconfigファイルを読ませるために、カレントフォルダにrtc.confを事前に作成しておく。

* コマンドラインオプション "-f"
* 環境変数 "RTC_MANAGER_CONFIG"
* デフォルト設定ファイル "./rtc.conf"
* デフォルト設定ファイル "/etc/rtc.conf"
* デフォルト設定ファイル "/etc/rtc/rtc.conf"
* デフォルト設定ファイル "/usr/local/etc/rtc.conf"
* デフォルト設定ファイル "/usr/local/etc/rtc/rtc.conf"
* 埋め込みコンフィギュレーション値

In [5]:
!ls ./rtc.conf

./rtc.conf


rtc.confの内容は以下の通り。
logger.enableをYESに設定し起動履歴をログに出力する。Python上ではうまくエラーが拾えないので必須。

logger.file_nameで、ログ・ファイル名を指定。rtc.log に出力する設定とする。

In [6]:
!more ./rtc.conf

corba.nameservers: localhost
exec_cxt.periodic.rate:1000.0
naming.formats: %n.rtc
logger.enable: YES
logger.file_name: rtc.log


# RTCのソース

今回は、104のAngleをSPI通信で、SP3008というチップを経由して取得しにいくので、`import spidev`でspiのライブラリをimportする。　

In [1]:
#!/usr/bin/env python
# coding: utf-8
# Python 

import RPi.GPIO as GPIO
import spidev
import time
import sys
import RTC
import OpenRTM_aist

configurationの値を設定する。

In [2]:
fabobutton_spec = ["implementation_id", "FaBoButton",
              "type_name",      "FaBoButtonComponent",
              "description",      "FaBo Button component",
              "version",            "1.0",
              "vendor",            "GClue, Inc.",
              "category",          "fabo",
              "activity_type",     "DataFlowComponent",
              "max_instance",   "10",
              "language",          "Python",
              "lang_type",         "script",
              ""]

BUTTON_PIN = 5

Buttonが押されたらTrueを、戻ったらFalseを送る

In [3]:
class FaBoButton(OpenRTM_aist.DataFlowComponentBase):

    def __init__(self, manager):
        print "__init__"
        OpenRTM_aist.DataFlowComponentBase.__init__(self, manager)

        self._button     = RTC.TimedBoolean(RTC.Time(0,0),False)
        self._buttonOut    = OpenRTM_aist.OutPort("BUTTON", self._button)
        self.addOutPort("BUTTON", self._buttonOut)

        return

    def onInitialize(self):
        print "onInitialize"

        GPIO.setwarnings(False)
        GPIO.setmode( GPIO.BCM )
        GPIO.setup(BUTTON_PIN, GPIO.IN )
        return RTC.RTC_OK

    def onExecute(self, ec_id):
        print "onExcute"

        if( GPIO.input(BUTTON_PIN)):
            self._button.data = True
            self._buttonOut.write()
        else:
            self._button.data = False
            self._buttonOut.write()

        return RTC.RTC_OK

In [4]:
def FaBoButtonInit(manager):
    print "FaBoButtonInit"
    profile = OpenRTM_aist.Properties(defaults_str=fabobutton_spec)
    manager.registerFactory(profile,
                            FaBoButton,
                            OpenRTM_aist.Delete)    

def MyModuleInit(manager):
    print "MyModuleInit"
    FaBoButtonInit(manager)

    # Create a component
    comp = manager.createComponent("FaBoButton")

def main():
    print "main"
    mgr = OpenRTM_aist.Manager.init(sys.argv)
    mgr.setModuleInitProc(MyModuleInit)
    mgr.activateManager()
    mgr.runManager()

if __name__ == "__main__":
    main()

main
MyModuleInit
FaBoButtonInit
__init__
onInitialize
Traceback (most recent call last):
  File "/usr/lib/python2.7/dist-packages/OpenRTM_aist/CorbaNaming.py", line 616, in unbind
    self._rootContext.unbind(name_)
  File "/usr/lib/python2.7/dist-packages/CosNaming_idl.py", line 234, in unbind
    return _omnipy.invoke(self, "unbind", _0_CosNaming.NamingContext._d_unbind, args)
NotFound: CosNaming.NamingContext.NotFound(why=missing_node, rest_of_name=[CosNaming.NameComponent(id='FaBoButtonComponent0', kind='rtc')])

